In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
%cd "/content/drive/MyDrive/Colab Notebooks/AI_graduation_project/trainer_Retro-Reader"

/content/drive/MyDrive/Colab Notebooks/AI_graduation_project/trainer_Retro-Reader


* ### to_csv

In [72]:
!pip install datasets
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
import pandas as pd
import json
from tqdm import tqdm

from typing import Union, Any, Dict
from datasets.arrow_dataset import Batch
from datasets import Dataset

import argparse
import datasets

In [74]:
def load_and_convert(filepath):
    squad_dataset = pd.DataFrame(columns=["id", "title", "context", "question", "answers"])
    with open(filepath) as f:
        squad = json.load(f)
        for article in tqdm(squad["data"]): # {title, paragraphs}
            title = article.get("title", "").strip()
            for paragraph in article["paragraphs"]: # paragraph : {qas, context}
                context = paragraph["context"].strip()

                for qa in paragraph["qas"]: 
                # ans_qa : {question, id, answers=[{text, answer_start}], is_impossible}
                # unans_qa : {plausible_answer=[{text, answer_start}], question, id, answers=[], is_impossible}
                    question = qa["question"].strip()
                    id_ = qa["id"]
                    if qa["answers"]:
                        answer_starts = [answer["answer_start"] for answer in qa["answers"]]
                        answers = [answer["text"].strip() for answer in qa["answers"]]
                    else:
                        answer_starts = []
                        answers = []
                    squad_dataset.loc[len(squad_dataset)] = [id_, title, context, question, {"text": answers, "answer_start": answer_starts}]
    return squad_dataset

In [75]:
train_df = load_and_convert("./after_processing_data/Training/training_12m.json")
valid_df = load_and_convert("./after_processing_data/Validation/validation_12m.json")

100%|██████████| 12840/12840 [00:40<00:00, 320.43it/s]


In [76]:
train_df.to_csv("./after_processing_data/Training/f_training_12m.csv", mode='w')
valid_df.to_csv("./after_processing_data/Validation/f_validation_12m.csv", mode='w')

* ### 전처리 2차

In [77]:
# 파일명 수정 (1)

# train_df = pd.read_csv("data/df_train_10_pre11.csv")
# dev_df = pd.read_csv("data/df_dev_10_pre11.csv")
train_df = pd.read_csv("./after_processing_data/Training/f_training_12m.csv")
dev_df = pd.read_csv("./after_processing_data/Validation/f_validation_12m.csv")

- [ ]  이후에 전처리 방향 결정해서, 30 epoch 학습
- [ ]  결과보고, 전처리1 or 2 결정해서 가원님께 전달, dataset 늘려서 학습.
- [ ]  깃허브 올리기
- [ ]  30 epoch 모델 전달.

1) 특수기호 추출하기

In [78]:
full_df = pd.concat([train_df, dev_df])

In [79]:
tmp_train = train_df['context'][:30]

In [80]:
# 첫 단어 첫번째 문자 확인 후 symbols 에 append
# 추가) 결과 관찰 도중 ] } 처럼 닫는 특수기호들이 추출이 안되는 경우 발생
#     ) 마지막 단어 확인 후 append도 추가함.

import re
symbols = []
pattern = re.compile('[^A-Za-z0-9가-힣]') 
for idx, context in enumerate(full_df['context']):
  # if idx == 5:
  #   break
  for line in context.split("\n"):
    if len(line.split()) > 0:
      first = line.split()[0]
      last = line.split()[-1]
      f = pattern.findall(first)
      l = pattern.findall(last)
      if bool(f): # 첫번째 단어의 특수문자
        symbols += f 
      if bool(l): # 마지막 단어의 특수문자
        symbols += l

In [81]:
from collections import Counter

result = Counter(symbols)
# print(result.most_common())

In [82]:
result = dict(result.most_common()).keys()

2) 특수기호 제거

2-1) answer에서 제외할 특수기호 추출

In [83]:
full_df['answers'].head()

0    {'text': ['금55,570,000원'], 'answer_start': [210]}
1    {'text': ['금55,570,000원'], 'answer_start': [210]}
2             {'text': ['11대'], 'answer_start': [189]}
3             {'text': ['11대'], 'answer_start': [189]}
4              {'text': ['4대'], 'answer_start': [155]}
Name: answers, dtype: object

In [84]:
import re
answer_symbols = []
pattern = re.compile('[^A-Za-z0-9가-힣\s]') 
for idx, ans in enumerate(full_df['answers']):
  ans = eval(ans)['text']
  # if idx == 1000:
  #   break
  if len(ans) == 0:
    continue
  check = pattern.findall(ans[0])
  if bool(check):
    answer_symbols += check
    # print(ans) # 생각보다 answers에 특수기호가 정말 많다는 걸 알게됨...

In [85]:
answer_result = Counter(answer_symbols)

print(answer_result.most_common())
print("\n\n")
answer_symbol_list = list(dict(answer_result.most_common()).keys())
print(answer_symbol_list)

[(',', 26912), ('.', 13184), (')', 13013), ('(', 12932), ('·', 4766), ('‘', 3458), ('’', 3443), ('-', 3355), ('%', 2966), ('․', 2828), ('*', 1964), (':', 1450), ('/', 1379), ('~', 1106), ('&', 999), ('→', 999), ('‧', 884), ('」', 820), ('「', 750), ('○', 686), ('“', 614), ('”', 608), ('ㅇ', 491), ('ㆍ', 405), ('②', 237), ('｣', 231), ("'", 231), ('｢', 222), ('～', 219), ('①', 216), ('▲', 215), ('㈜', 194), ('③', 189), ('⋅', 187), ('❍', 175), ('+', 172), ('∼', 170), ('△', 170), ('･', 169), ('』', 118), ('⇒', 117), ('㎡', 116), ('>', 113), ('『', 113), ('・', 90), ('◦', 88), ('④', 83), ('※', 69), ('□', 63), ('×', 62), ('\uf000', 61), ('!', 60), ('<', 60), ('℃', 58), ('[', 48), (']', 48), ('㎒', 40), ('▴', 38), ('•', 33), ('"', 33), ('–', 32), ('➡', 30), ('⑤', 30), ('☎', 30), ('㎞', 28), ('▶', 27), ('㎓', 24), ('⇨', 22), ('▪', 21), ('∙', 20), (';', 19), ('⑥', 18), ('▵', 18), ('➋', 18), ('#', 18), ('$', 17), ('▸', 16), ('〜', 16), ('=', 15), ('?', 15), ('‣', 14), ('Ⅱ', 14), ('_', 14), ('▹', 14), ('☞', 13

In [86]:
answer = ""
for a in answer_symbol_list:
  answer += a

In [87]:
import string
answer.replace(string.punctuation, '')

',.)(·‘’-%․*:/~&→‧」「○“”ㅇㆍ②｣\'｢～①▲㈜③⋅❍+∼△･』⇒㎡>『・◦④※□×\uf000!<℃[]㎒▴•"–➡⑤☎㎞▶㎓⇨▪∙;⑥▵➋#$▸〜=?‣Ⅱ_▹☞°➊㎥㎢◯㎝@➀➁➂ᆞ↔➌❶❷❸❹Ⅰ·\u200bγ■◈㎾㎍√é❒〔〕ⅰⅱ➍〉－❏\xad²⌟ⅲ↑±❺◊—Ⅲ【】▢☏⌜♣▷ㅋㄷ年⟶ℓμᄋ⑦㎐㉮㉯㉰ᐧ{}‐ⅳⅴ㎑¦金∘識➃➄➅\U000f02b3↓＊＆\\§☆➎₂ê㎘❻￮㎎⃞㎿̇ʻʼⅤβ㎚÷㎜㏊《》⁕ⓑⓒ兩₄ｓ₆臨＋ⅵⅶα更㎏★☓㉱來ｇ'

확인해본 결과)
- 반려인ㆍ반려견
- ~ 
이 많이 나오는데 같은 의미로 사용되는 것들이 다양함.  
answer쪽 특수문자 분석 결과 단위가 되는 것 위주로 제외하기로 함.

In [88]:
answer_except = "±㎘㎥㏊～∼㎡㈜㎞℃〜°㎝金年êｓ"

2-2) context에서 제외할 특수기호 추출

In [89]:
symbol_list = list(result)

In [90]:
symbol = ""
for s in symbol_list:
  symbol += s 
print(symbol)

.*-□○)ㅇ,(❍※◦·「%><․Ⅱ‘“”②Ⅰ①Ⅲ!~:」/?③’]Ⅳ▢】【▪[‧•◈④◇『→‣｢⇒▸△』☞∙～･Ⅴ⑤■↑▶ㆍ●∼&+㎡'－＊∘❖▲◊❚➊《⇨◯》➋⑥⋅☐㈜｣◆❏="➌〇­▣Ⅵ❑ᆞ▷・⑦❶❷≪≫–▹×❸〕⑧↓〉〈◉§〔⃞➡➁⦁Ⅶ➀❹⑨¦▴￭➍★√_㎥◾➂❒֠☎⑩㎞▫#⧠◎◌‑ᄋㆁо…ⅱ⁃󰊱℃@ⅰ✔〜㎢❰❱󰊲$⚪⧈—➎➃➠￮►Ⅷ∐❐¡✓ⅲ󰊳◻￦ｏ⑪㎍✤―▮±❺：✽◕▌‐▵➄⑴➤​⑵}♣㎒⧭◀➟！ℓ|〚➭㎓㎏⌜α㎎▆Ⅸᐅ㎜➏⑶{ⅳ➅−⁕◼;⑫１⟹″✍＞╺󰊴¨₩�‌⑷੦↔┗⁍\❻ʻŒ）∣➜↳⟶＜⑬⑮☏➯㉰➪⚬〛◐°Ÿ⑭＃²⇀₂﹡ⓛ㉮㉯β⋄󰋮·é➲➨✜▍➫∎㎝⁎㎖⭘☑≤∆ⅴＯ‚³⁺Ⅻ⃟❃ᐧ́‥㉱➐㎾［］Ž⑯⑱ㅡ✥ㅁ⌟┖ü⃝㉲㉳└➛ⓐⓑ⨯＋󰊶．ƒ＂≥ⅵⅶμ▯♬の➢▻◤◢⁜Ⅹ㉠㉡÷▼′‶󰋎󰋏㏊⑰⑳㉔㉕㉖㉗㉘㉙㉚◁󰋯➒´󰊵➆┎➝２㉴㉵⇑ㅣ❳㎠┞⚫Ｘ⧬➞‰✲❙٠㈔，㎕†☀✉陵❯⇢˙„⑲㉑㉒㉓￫㈀㈁㈂󰡓Ö▽€老金％➑≒㎧㎿⇛⑻φ㏄®＇年̇❼❽♤♠≃├█㉢ㄴ（⑸Ⅺ➓⓶∽㎘ꊲ˚˝


In [91]:
import string
symbol = symbol.replace('-―‌∙“”·‘’․~‑@%/#{}:;*.∼~～=&×㎢㎒㎎㎜㎍‰㎏㎓㎧㈔μ㏊' + string.punctuation + answer_except, '')

In [92]:
# # 제외할 특수문자
# print(symbol)

In [93]:
symbol_final = symbol.translate(str.maketrans('','', '-―‌∙“”·‘’․~‑@%/#{}:;*.∼~～=&×㎢㎒㎎㎜㎍‰㎏㎓㎧㈔μ㏊' + string.punctuation + answer_except))

In [94]:
# print(symbol + '\n')
# print(symbol_final)

In [95]:
def delete_symbols(con, mode='context'):
  con_ = ""
  for x in con.split('\n'):
    # 특수문자 제거
    x = x.translate(str.maketrans('','', symbol_final))
    
    # '-' 가장 첫 번째, 마지막 단어 제거
    if x.split()[0] == '-': # 첫 번째 단어 - 일 때, 첫 단어 '-'만 제거
      idx = x.find('-')
      x = x[:idx] + x[idx+1:] # 마지막 단어 '-'만 제거
      if x.split()[-1] == '-':
        idx = x.rfind('-')
        x = x[:idx] + x[idx+1:]

    # 개행 '. ' 으로 대체  
    if mode=='answer':
      return x
    con_ += x + '. ' 
  return con_

3) answer 처리

In [96]:
train_df['answers'][0]

"{'text': ['금55,570,000원'], 'answer_start': [210]}"

In [97]:
ans = eval(train_df['answers'][0])
answer_text = delete_symbols(ans['text'][0], mode='answer')
answer_text

'금55,570,000원'

4) 최종

In [98]:
# 파일명 수정 (2)
train_df = pd.read_csv("./after_processing_data/Training/f_training_12m.csv")
dev_df = pd.read_csv("./after_processing_data/Validation/f_validation_12m.csv")

In [99]:
def delete_symbols(con, mode='context'):
  con_ = ""
  for x in con.split('\n'):
    # 특수문자 제거
    x = x.translate(str.maketrans('','', symbol_final))
    
    # '-' 가장 첫 번째, 마지막 단어 제거
    if len(x.split()) > 0:
      if x.split()[0] == '-': # 첫 번째 단어 - 일 때, 첫 단어 '-'만 제거
        idx = x.find('-')
        x = x[:idx] + x[idx+1:] # 마지막 단어 '-'만 제거
        if len(x.split())>0 and x.split()[-1] == '-':
          idx = x.rfind('-')
          x = x[:idx] + x[idx+1:]

    # 개행 '. ' 으로 대체  
    if mode=='answer':
      return x
    con_ += x + '. ' 
  return con_

In [100]:
# train_df = train_df['context'].apply(delete_symbols)

for i, x in enumerate(train_df['context']):
  train_df.loc[i, 'context'] = delete_symbols(x)

for i, x in enumerate(train_df['answers']):
  x = eval(x)
  if len(x['text']) > 0:
    answer_text = delete_symbols(x['text'][0], mode='answer')
    index = train_df['context'][i].find(answer_text)
    train_df.loc[i, 'answers'] = str({'text': [answer_text], 'answer_start': [index]})

for i, x in enumerate(dev_df['context']):
  dev_df.loc[i, 'context'] = delete_symbols(x)

for i, x in enumerate(dev_df['answers']):
  x = eval(x)
  if len(x['text']) > 0:
    answer_text = delete_symbols(x['text'][0], mode='answer')
    index = dev_df['context'][i].find(answer_text)
    dev_df.loc[i, 'answers'] = str({'text': [answer_text], 'answer_start': [index]})

In [101]:
# train_df.to_csv('data/df_train_10_prep2.csv')
# dev_df.to_csv('data/df_dev_10_prep2.csv')
train_df.to_csv('./after_processing_data/Training/f_training_12m.csv')
dev_df.to_csv('./after_processing_data/Validation/f_validation_12m.csv')